# Script for scCODA analysis
### Description:
This script takes as input the table of total count of cells for each cell type per bat individual. scCODA analysis is then performed to determine which cell types are differentially abundant across ages. All pair-wise tests are performed (Adu vs Juv, Juv vs Adu, Adu vs Sub...). Only significant differences with FDR < 0.05 or 0.1 were further considered.

In [ ]:
# Setup python packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import pickle as pkl
import matplotlib.pyplot as plt
import seaborn as sns
import arviz as az

from sccoda.util import comp_ana as mod
from sccoda.util import cell_composition_data as dat
from sccoda.util import data_visualization as viz

## Load data from input table

In [ ]:
# Load data
cell_counts = pd.read_csv("../outputs/scCODA/Input_table_scCODA.csv")
print(cell_counts)

In [ ]:
# Convert data to anndata object
data_all = dat.from_pandas(cell_counts, covariate_columns=["Age", "Individual"])
print(data_all)

In [ ]:
# Boxplot 
viz.boxplots(data_all, feature_name="Age")

In [ ]:
# Stacked barplot per Age
viz.stacked_barplot(data_all, feature_name="Age")


# Stacked barplot per Individual
viz.stacked_barplot(data_all, feature_name="Individual")

## Build scCODA Model with Adult individuals as default

In [ ]:
# Model setup and inference
model_age_Adu = mod.CompositionalAnalysis(data_all,
                                          formula="C(Age, Treatment('Adult'))",
                                          reference_cell_type="automatic")

# Run MCMC
sim_results_Adu = model_age_Adu.sample_hmc(num_results=50000, num_burnin=10000)

## Build scCODA Model with Subadult individuals as default

In [ ]:
# Model setup and inference
model_age_Sub = mod.CompositionalAnalysis(data_all,
                                          formula="C(Age, Treatment('Subadult'))",
                                          reference_cell_type="automatic")

# Run MCMC
sim_results_Sub = model_age_Sub.sample_hmc(num_results=50000, num_burnin=10000)

## Build scCODA Model with Juvenile individuals as default

In [ ]:
# Model setup and inference
model_age_Juv = mod.CompositionalAnalysis(data_all,
                                          formula="C(Age, Treatment('Juvenile'))", 
                                          reference_cell_type="automatic")

# Run MCMC
sim_results_Juv = model_age_Juv.sample_hmc(num_results=50000, num_burnin=10000)

## Save results

In [ ]:
# Save scCODA results
sim_results_Adu.save("../outputs/scCODA/results_Adu")
sim_results_Sub.save("../outputs/scCODA/results_Sub")
sim_results_Juv.save("../outputs/scCODA/results_Juv")

## Load results and explore manually 

In [ ]:
# Play with these two variables (path / chosen_fdr) to explore each age

path = "../outputs/scCODA/results_Sub"
chosen_fdr = 0.1 #Should check results with FDR 0.05 and 0.1


with open(path, "rb") as f:
    sim_results_loaded = pkl.load(f)
    
sim_results_loaded.set_fdr(est_fdr= chosen_fdr) #select FDR for loaded data
print(sim_results_loaded.credible_effects()) #display credible results

## Load results and save them as separated files for analysis in R

In [ ]:
# Save all results as .csv tables for each FDR
path = "../outputs/scCODA/"
result_files = ["results_Adu", "results_Sub", "results_Juv"]
fdr_thresholds = [0.1, 0.05]

for x in result_files:
    for y in fdr_thresholds:
          with open(path + x, "rb") as f:
            sim_results_loaded = pkl.load(f)
    
            sim_results_loaded.set_fdr(est_fdr= y) #select FDR for loaded data
            table = sim_results_loaded.credible_effects() #table with credible results
            table.to_csv(path + "table_" + x + "_FDR_" + str(y) + ".csv")